# 설정

In [1]:
# 파이썬 ≥ 3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥ 0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Support Vector Machine"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

결정 경계를 그려주는 함수

In [2]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, text_idx=None, resolution=0.02):
    
    # 마커와 컬러맵을 설정한다
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # 결정 경계를 그린다
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.8,
                    c=colors[idx],
                    marker=markers[idx],
                    label=cl,
                    edgecolor='black')
        
    # 테스트 샘플을 부각하여 그린다
    if test_idx:
        X_test, y_test = X[test_idx, :], y[test_idx]
        
        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    facecolors='none',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100,
                    label='test set')

다음 코드는 iris 데이터를 불러들이고 학습셋과 테스트셋으로 나눈 후 표준화를 하는 과정이다.

In [3]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print('클래스 레이블:', np.unique(y))

클래스 레이블: [0 1 2]


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

# 서포트 벡터 머신을 사용한 최대 마진 분류

<b>서포트 벡터 머신</b><sup>Support Vector Machine, SVM</sup>은 강력하고 널리 사용되는 학습 알고리즘이다. SVM은 퍼셉트론의 확장으로 생각할 수 있다. 앞서 퍼셉트론 알고리즘을 사용하여 분류 오차를 최소화했다. SVM의 최적화 대상은 마진을 최대화하는 것이다. 마진은 클래스를 구분하는 초평면(결정 경계)과 이 초평면에 가장 가까운 훈련 샘플 사이의 거리로 정의한다. 이런 샘플을 <b>서포터 벡터</b><sup>support vector</sup>라고 한다. [그림 1]에 나타나 있다.

<b>그림 1</b> 서포트 벡터 머신
<div style="text-align:center;">
    <img src="./images/Support Vector Machine/서포트 벡터 머신.png">
</div>

## 최대 마진

큰 마진<sup>large margin</sup>의 결정 경계를 원하는 이유는 일반화 오차가 낮아지는 경향이 있기 때문이다. 반면에 작은 마진의 모델은 과대적합되기 쉽다. 마진 최대화를 이해하기 위해 결정 경계와 나란히 놓인 양성 샘플 쪽의 초평면과 음성 샘플 쪽의 초평면을 자세히 살펴보겠다. 이 두 초평면은 다음과 같이 쓸 수 있다.<sup><a id="a01" href="#p01">[1]</a></sup>

<div style="text-align:center;">
    <img src="./images/formula/Support Vector Machine/양쪽 초평면 식.png">
</div>

두 선형식 (1)과 (2)를 빼면 다음 결과를 얻는다.

<div style="text-align:center;">
    <img src="./images/formula/Support Vector Machine/양쪽 초평면 식2.png">
</div>

이 식을 다음과 같은 벡터 w의 길이로 정규화할 수 있다.

<div style="text-align:center;">
    <img src="./images/formula/Support Vector Machine/w의 길이.png">
</div>

결과 식은 다음과 같다.

<div style="text-align:center;">
    <img src="./images/formula/Support Vector Machine/초평면 사이의 거리.png">
</div>

이 식의 좌변은 양성 쪽 초평면과 음성 쪽 초평면 사이의 거리로 해석할 수 있다.<sup><a id="a02" href="#p02">[2]</a></sup> 이것이 최대화하려고 하는 <b>마진</b><sup>margin</sup>이다.

# 미주

<b id="p01">1</b> 여기서는 편의상 <i>w</i><sub>0</sub>를 따로 표기했다. 이 식은 <i>w</i><sub>2</sub>축과 만나는 절편을 1 / <i>w</i><sub>2</sub> 만큼 위아래로 결정 경계와 나란하게 이동한 직선이 된다. [↩](#a01)

<b id="p02">2</b> 직선 Ax+By+c=0과 점(x<sub>0</sub>, y<sub>0</sub>) 사이의 거리 <i>d</i>=(Ax<sub>0</sub>+By<sub>0</sub>+c)/(√(A<sup>2</sup>+B<sup>2</sup>) 이므로 왼쪽 식은 두 서포트 벡터 x<sub>pos</sub>와 x<sub>neg</sub> 사이의 거리가 된다. [↩](#a02)